# Functional Analysis — Hilbert Spaces & Spectral Theory

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 225-252
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# Standard scientific stack
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import sympy as sp
from scipy import integrate, linalg
from scipy.special import hermite
from numpy.polynomial.legendre import leggauss

%matplotlib inline

plt.rcParams.update({
    "figure.figsize": (10, 7),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "lines.linewidth": 2,
    "legend.fontsize": 11,
    "font.family": "serif",
    "figure.dpi": 120,
})
print("Imports ready — numpy, matplotlib, sympy, scipy loaded.")

\
## 1. Metric Spaces and Convergence

A **metric space** $(X, d)$ is a set $X$ with a distance function $d: X \times X \to \mathbb{R}$ satisfying:

1. $d(x,y) \geq 0$ with $d(x,y)=0 \iff x=y$ (positivity)
2. $d(x,y) = d(y,x)$ (symmetry)
3. $d(x,z) \leq d(x,y) + d(y,z)$ (triangle inequality)

A sequence $\{x_n\}$ **converges** to $x$ if $d(x_n, x) \to 0$ as $n \to \infty$.

A metric space is **complete** if every Cauchy sequence converges. This is
critical: a **Hilbert space** is a *complete* inner product space.

### Examples of Metrics

| Space | Metric | Complete? |
|-------|--------|-----------|
| $\mathbb{R}^n$ | $d(x,y) = \|x-y\|_2$ | Yes |
| $\mathbb{Q}$ | $d(x,y) = |x-y|$ | **No** ($\sqrt{2}$ is a limit point not in $\mathbb{Q}$) |
| $C[0,1]$ | $d(f,g) = \max|f-g|$ | Yes (uniform convergence) |
| $L^2[0,1]$ | $d(f,g) = \sqrt{\int|f-g|^2}$ | Yes (Riesz-Fischer theorem) |

In [ ]:
\
# Metric spaces: visualize sequence convergence and Cauchy sequences

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Left: Converging sequence in R ---
ax = axes[0]
n = np.arange(1, 31)
# Sequence: x_n = 1 + (-1)^n / n  → converges to 1
x_n = 1 + (-1)**n / n
ax.plot(n, x_n, 'bo-', markersize=5, label='$x_n = 1 + (-1)^n/n$')
ax.axhline(1, color='red', ls='--', lw=2, label='Limit = 1')
ax.fill_between(n, 1 - 1/n, 1 + 1/n, alpha=0.1, color='green', label='$\\epsilon$-band')
ax.set_xlabel("$n$", fontsize=12)
ax.set_ylabel("$x_n$", fontsize=12)
ax.set_title("Convergent Sequence in $\\mathbb{R}$", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# --- Middle: Cauchy sequence approximating √2 in Q ---
ax2 = axes[1]
# Newton's method: x_{n+1} = (x_n + 2/x_n)/2
x = [1.0]
for _ in range(10):
    x.append((x[-1] + 2/x[-1]) / 2)
x = np.array(x)
errors = np.abs(x - np.sqrt(2))

ax2.semilogy(range(len(x)), errors, 'ro-', markersize=8, label='$|x_n - \\sqrt{2}|$')
ax2.set_xlabel("Iteration $n$", fontsize=12)
ax2.set_ylabel("Error", fontsize=12)
ax2.set_title("Cauchy Seq. in $\\mathbb{Q}$: $x_{n+1}=(x_n+2/x_n)/2$", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1e-16, 10)

# --- Right: Function space convergence ---
ax3 = axes[2]
x_plot = np.linspace(0, 1, 500)
colors = plt.cm.viridis(np.linspace(0, 1, 8))
for k, c in zip(range(1, 9), colors):
    f_n = x_plot**k
    ax3.plot(x_plot, f_n, color=c, lw=1.5, label=f'$f_{k}(x) = x^{k}$' if k <= 4 else None)

# Limit function (pointwise)
f_limit = np.zeros_like(x_plot)
f_limit[-1] = 1
ax3.plot(x_plot, f_limit, 'r--', lw=3, label='Pointwise limit')

ax3.set_xlabel("$x$", fontsize=12)
ax3.set_ylabel("$f_n(x) = x^n$", fontsize=12)
ax3.set_title("Pointwise vs Uniform Convergence", fontsize=13)
ax3.legend(fontsize=9, loc='upper left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Left: Convergent sequence oscillates within shrinking ε-band.")
print("Middle: Newton iteration converges quadratically (Cauchy in Q, but √2 ∉ Q).")
print("Right: x^n → 0 pointwise on [0,1), but limit is discontinuous — not uniform convergence.")

\
## 2. The $L^2$ Space and Inner Products

The space $L^2[a,b]$ consists of square-integrable functions:

$$L^2[a,b] = \left\{f : \int_a^b |f(x)|^2\,dx < \infty\right\}$$

with **inner product**:

$$\langle f, g \rangle = \int_a^b f^*(x)\,g(x)\,dx$$

and **norm**: $\|f\| = \sqrt{\langle f, f \rangle}$.

### Properties

- **Cauchy-Schwarz:** $|\langle f,g\rangle| \leq \|f\|\cdot\|g\|$
- **Triangle inequality:** $\|f+g\| \leq \|f\| + \|g\|$
- **Orthogonality:** $f \perp g \iff \langle f,g\rangle = 0$
- **Pythagorean theorem:** If $f \perp g$, then $\|f+g\|^2 = \|f\|^2 + \|g\|^2$

$L^2$ is the natural home of quantum mechanics: **wavefunctions live in $L^2$**.

In [ ]:
\
# L² inner product: demonstrate orthogonality of sine functions

from scipy.integrate import quad

def L2_inner_product(f, g, a=0, b=2*np.pi):
    '''Compute <f, g> = integral_a^b f*(x) g(x) dx.'''
    real_part, _ = quad(lambda x: np.real(np.conj(f(x)) * g(x)), a, b)
    imag_part, _ = quad(lambda x: np.imag(np.conj(f(x)) * g(x)), a, b)
    return real_part + 1j * imag_part

def L2_norm(f, a=0, b=2*np.pi):
    '''Compute ||f|| = sqrt(<f, f>).'''
    return np.sqrt(np.real(L2_inner_product(f, f, a, b)))

# Orthonormality of {sin(nx)/√π, cos(nx)/√π} on [0, 2π]
print("Orthogonality of Fourier basis on [0, 2π]")
print("=" * 55)

# Check ⟨sin(mx), sin(nx)⟩ for m, n = 1..4
print("\n⟨sin(mx), sin(nx)⟩:")
for m in range(1, 5):
    row = []
    for n in range(1, 5):
        ip = np.real(L2_inner_product(
            lambda x, m=m: np.sin(m*x),
            lambda x, n=n: np.sin(n*x)
        ))
        row.append(f"{ip:7.3f}")
    print(f"  m={m}: " + " ".join(row))

# Check ⟨sin(mx), cos(nx)⟩
print("\n⟨sin(mx), cos(nx)⟩:")
for m in range(1, 5):
    row = []
    for n in range(1, 5):
        ip = np.real(L2_inner_product(
            lambda x, m=m: np.sin(m*x),
            lambda x, n=n: np.cos(n*x)
        ))
        row.append(f"{ip:7.3f}")
    print(f"  m={m}: " + " ".join(row))

# Norms
print("\nNorms:")
for n in range(1, 5):
    norm_sin = L2_norm(lambda x, n=n: np.sin(n*x))
    norm_cos = L2_norm(lambda x, n=n: np.cos(n*x))
    print(f"  ||sin({n}x)|| = {norm_sin:.6f} (expected √π = {np.sqrt(np.pi):.6f})")

# Cauchy-Schwarz inequality demonstration
f = lambda x: np.exp(-x/2)
g = lambda x: np.sin(x)
ip_fg = abs(L2_inner_product(f, g, 0, 2*np.pi))
nf = L2_norm(f, 0, 2*np.pi)
ng = L2_norm(g, 0, 2*np.pi)
print(f"\nCauchy-Schwarz: |⟨f,g⟩| = {ip_fg:.6f} ≤ ||f||·||g|| = {nf*ng:.6f}  ✓")

\
## 3. Fourier Basis as Orthonormal Set

The functions $\{e_n(x) = e^{inx}/\sqrt{2\pi}\}_{n=-\infty}^{\infty}$ form a
**complete orthonormal basis** for $L^2[0, 2\pi]$:

$$\langle e_m, e_n \rangle = \delta_{mn}$$

Any $f \in L^2$ can be expanded:

$$f(x) = \sum_{n=-\infty}^{\infty} c_n\, e_n(x), \qquad
  c_n = \langle e_n, f \rangle = \frac{1}{\sqrt{2\pi}}\int_0^{2\pi} e^{-inx} f(x)\,dx$$

**Parseval's theorem** (generalized Pythagorean theorem):

$$\|f\|^2 = \sum_{n=-\infty}^{\infty} |c_n|^2$$

This is the mathematical foundation of **quantum superposition**: any state
can be expanded in an orthonormal basis, and probabilities sum to 1.

In [ ]:
\
# Fourier decomposition and reconstruction of functions in L²

def fourier_coefficients(f, N, L=2*np.pi):
    '''Compute Fourier coefficients c_n for n = -N..N.'''
    coeffs = {}
    for n in range(-N, N+1):
        re, _ = quad(lambda x: np.real(f(x) * np.exp(-1j*n*x)), 0, L)
        im, _ = quad(lambda x: np.imag(f(x) * np.exp(-1j*n*x)), 0, L)
        coeffs[n] = (re + 1j*im) / L
    return coeffs

def fourier_reconstruct(coeffs, x, L=2*np.pi):
    '''Reconstruct f(x) from Fourier coefficients.'''
    result = np.zeros_like(x, dtype=complex)
    for n, cn in coeffs.items():
        result += cn * np.exp(1j * n * x)
    return result

# Test functions
x = np.linspace(0, 2*np.pi, 1000)

# Square wave
def square_wave(x):
    return np.where(np.mod(x, 2*np.pi) < np.pi, 1.0, -1.0)

# Sawtooth
def sawtooth(x):
    return (np.mod(x, 2*np.pi) - np.pi) / np.pi

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for row, (fname, f) in enumerate([(r"Square wave", square_wave), (r"Sawtooth", sawtooth)]):
    ax_left = axes[row, 0]
    ax_right = axes[row, 1]

    # Plot original and approximations
    ax_left.plot(x, f(x), 'k-', lw=2, label='Original')
    N_values = [1, 3, 7, 20]
    colors = ['red', 'orange', 'green', 'blue']

    all_coeffs = {}
    for N, color in zip(N_values, colors):
        coeffs = fourier_coefficients(f, N)
        recon = np.real(fourier_reconstruct(coeffs, x))
        ax_left.plot(x, recon, color=color, lw=1.5, alpha=0.7, label=f'N={N}')
        all_coeffs[N] = coeffs

    ax_left.set_title(f"Fourier Reconstruction: {fname}", fontsize=13)
    ax_left.set_xlabel("$x$")
    ax_left.set_ylabel("$f(x)$")
    ax_left.legend(fontsize=9)
    ax_left.grid(True, alpha=0.3)

    # Power spectrum (Parseval)
    N_big = 30
    coeffs_big = fourier_coefficients(f, N_big)
    ns = sorted(coeffs_big.keys())
    powers = [abs(coeffs_big[n])**2 for n in ns]

    ax_right.stem(ns, powers, linefmt='b-', markerfmt='bo', basefmt='gray')
    ax_right.set_title(f"Power Spectrum $|c_n|^2$: {fname}", fontsize=13)
    ax_right.set_xlabel("$n$ (frequency)")
    ax_right.set_ylabel("$|c_n|^2$")
    ax_right.grid(True, alpha=0.3)

    # Verify Parseval's theorem
    total_power = sum(powers)
    norm_sq = quad(lambda x: f(x)**2, 0, 2*np.pi)[0] / (2*np.pi)
    print(f"{fname}: Σ|cₙ|² = {total_power:.6f}, ||f||²/(2π) = {norm_sq:.6f}, "
          f"ratio = {total_power/norm_sq:.6f}")

plt.tight_layout()
plt.show()
print("\nParseval's theorem: energy in function = sum of energies in Fourier modes.")
print("This is the mathematical basis of quantum probability: Σ|cₙ|² = 1.")

\
## 4. Operators on Hilbert Space

A **linear operator** $\hat{A}: \mathcal{H} \to \mathcal{H}$ maps vectors to vectors.
In finite dimensions, operators are represented by **matrices**.

### The Adjoint

The **adjoint** $\hat{A}^\dagger$ satisfies:

$$\langle \hat{A}^\dagger \psi, \phi \rangle = \langle \psi, \hat{A}\phi \rangle$$

In matrix representation: $A^\dagger = \overline{A}^T$ (conjugate transpose).

### Classification of Operators

| Type | Condition | Properties |
|------|-----------|-----------|
| **Self-adjoint (Hermitian)** | $\hat{A}^\dagger = \hat{A}$ | Real eigenvalues, orthogonal eigenvectors |
| **Unitary** | $\hat{U}^\dagger\hat{U} = \hat{I}$ | Preserves inner products |
| **Normal** | $[\hat{A}, \hat{A}^\dagger] = 0$ | Diagonalizable via spectral theorem |
| **Projection** | $\hat{P}^2 = \hat{P} = \hat{P}^\dagger$ | Projects onto subspace |

In quantum mechanics:
- **Observables** are self-adjoint operators
- **Time evolution** is a unitary operator $\hat{U}(t) = e^{-i\hat{H}t/\hbar}$
- **Measurements** are described by projection operators

In [ ]:
\
# Operators on finite-dimensional Hilbert space (matrices)

np.set_printoptions(precision=4, suppress=True)

# --- Pauli matrices (fundamental in quantum mechanics) ---
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
I2 = np.eye(2, dtype=complex)

print("Pauli Matrices — the operators of spin-1/2 quantum mechanics")
print("=" * 55)
for name, sigma in [("σ_x", sigma_x), ("σ_y", sigma_y), ("σ_z", sigma_z)]:
    print(f"\n{name} =")
    print(sigma)
    # Check Hermiticity
    is_hermitian = np.allclose(sigma, sigma.conj().T)
    # Eigenvalues
    eigenvalues = np.linalg.eigvalsh(sigma)
    # Trace and determinant
    tr = np.trace(sigma)
    det = np.linalg.det(sigma)
    print(f"  Hermitian: {is_hermitian}")
    print(f"  Eigenvalues: {eigenvalues.real}")
    print(f"  Trace: {tr.real:.0f}, Det: {det.real:.0f}")
    print(f"  σ² = I: {np.allclose(sigma @ sigma, I2)}")

# Verify algebraic relations
print("\nAlgebraic Relations:")
print(f"  [σ_x, σ_y] = 2iσ_z: {np.allclose(sigma_x@sigma_y - sigma_y@sigma_x, 2j*sigma_z)}")
print(f"  [σ_y, σ_z] = 2iσ_x: {np.allclose(sigma_y@sigma_z - sigma_z@sigma_y, 2j*sigma_x)}")
print(f"  [σ_z, σ_x] = 2iσ_y: {np.allclose(sigma_z@sigma_x - sigma_x@sigma_z, 2j*sigma_y)}")
print(f"  {{σ_x, σ_y}} = 0: {np.allclose(sigma_x@sigma_y + sigma_y@sigma_x, 0)}")

# Unitary operator: rotation about z-axis
theta = np.pi / 4
U = np.array([[np.exp(-1j*theta/2), 0],
              [0, np.exp(1j*theta/2)]])
print(f"\nRotation U(θ=π/4) about z-axis:")
print(f"  U†U = I: {np.allclose(U.conj().T @ U, I2)}")
print(f"  |det(U)| = 1: {abs(np.linalg.det(U)):.10f}")

\
## 5. Self-Adjoint Operators and the Spectral Theorem

### Key Properties of Self-Adjoint Operators

If $\hat{A} = \hat{A}^\dagger$, then:

1. **All eigenvalues are real:** $\hat{A}|\lambda\rangle = \lambda|\lambda\rangle \implies \lambda \in \mathbb{R}$
2. **Eigenvectors for distinct eigenvalues are orthogonal:** $\langle \lambda_i | \lambda_j \rangle = 0$ if $\lambda_i \neq \lambda_j$
3. **Eigenvectors form a complete basis** (spectral theorem)

### The Spectral Theorem

Any self-adjoint operator can be written as:

$$\hat{A} = \sum_n \lambda_n |\lambda_n\rangle\langle\lambda_n|$$

This is the **eigenvalue decomposition** — the operator is completely
determined by its eigenvalues and eigenvectors.

### Functional Calculus

For any function $f$, we can define $f(\hat{A})$:

$$f(\hat{A}) = \sum_n f(\lambda_n) |\lambda_n\rangle\langle\lambda_n|$$

This gives meaning to expressions like $e^{i\hat{H}t}$ — the time evolution operator.

In [ ]:
\
# Spectral theorem: diagonalize a Hermitian matrix and verify properties

# A physically motivated Hermitian matrix: tight-binding Hamiltonian
# Models electron hopping on a 1D chain (N sites, periodic boundary)
N = 8
t_hop = -1.0  # hopping parameter

# Construct Hamiltonian
H = np.zeros((N, N), dtype=complex)
for i in range(N):
    H[i, (i+1) % N] = t_hop
    H[(i+1) % N, i] = t_hop

print(f"Tight-binding Hamiltonian ({N}×{N}):")
print(H.real)

# Verify Hermiticity
print(f"\nH = H†: {np.allclose(H, H.conj().T)}")

# Diagonalize
eigenvalues, eigenvectors = np.linalg.eigh(H)

print("\nEigenvalues (energy levels):")
for i, lam in enumerate(eigenvalues):
    print(f"  E_{i} = {lam:.6f}")

# Verify orthonormality of eigenvectors
overlap = eigenvectors.conj().T @ eigenvectors
print(f"\nEigenvector orthonormality (V†V = I): {np.allclose(overlap, np.eye(N))}")

# Verify spectral decomposition: H = Σ λ_n |n⟩⟨n|
H_reconstructed = np.zeros((N, N), dtype=complex)
for n in range(N):
    vn = eigenvectors[:, n:n+1]
    H_reconstructed += eigenvalues[n] * (vn @ vn.conj().T)
print(f"Spectral decomposition H = ΣλₙPₙ: {np.allclose(H, H_reconstructed)}")

# Functional calculus: compute exp(iHt)
t = 0.5
U_spectral = np.zeros((N, N), dtype=complex)
for n in range(N):
    vn = eigenvectors[:, n:n+1]
    U_spectral += np.exp(1j * eigenvalues[n] * t) * (vn @ vn.conj().T)

U_matrix = linalg.expm(1j * H * t)
print(f"Functional calculus: exp(iHt) via spectral = via matrix exp: "
      f"{np.allclose(U_spectral, U_matrix)}")
print(f"exp(iHt) is unitary: {np.allclose(U_spectral.conj().T @ U_spectral, np.eye(N))}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Energy levels
ax = axes[0]
for i, E in enumerate(eigenvalues):
    ax.hlines(E, 0.3, 0.7, colors='blue', linewidth=2)
    ax.text(0.75, E, f'$E_{i}={E:.3f}$', fontsize=9, va='center')
ax.set_ylabel("Energy", fontsize=12)
ax.set_title(f"Tight-Binding Energy Spectrum (N={N})", fontsize=13)
ax.set_xticks([])
ax.grid(True, alpha=0.3, axis='y')

# Eigenvectors (probability density)
ax2 = axes[1]
sites = np.arange(N)
for i in [0, 1, N//2, N-1]:
    ax2.plot(sites, np.abs(eigenvectors[:, i])**2, 'o-',
             label=f'$|\\psi_{i}|^2$, E={eigenvalues[i]:.2f}')
ax2.set_xlabel("Site", fontsize=12)
ax2.set_ylabel("$|\\psi|^2$", fontsize=12)
ax2.set_title("Eigenvector Probability Densities", fontsize=13)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

# Exact eigenvalues vs theory: E_k = 2t cos(2πk/N)
ax3 = axes[2]
k = np.arange(N)
E_exact = 2 * t_hop * np.cos(2 * np.pi * k / N)
ax3.plot(sorted(E_exact), 'rs-', markersize=8, label='Exact: $2t\\cos(2\\pi k/N)$')
ax3.plot(eigenvalues, 'bo-', markersize=6, label='Numerical eigenvalues')
ax3.set_xlabel("Level index", fontsize=12)
ax3.set_ylabel("Energy", fontsize=12)
ax3.set_title("Numerical vs Exact Eigenvalues", fontsize=13)
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("All spectral theorem properties verified:")
print("  1. Real eigenvalues ✓")
print("  2. Orthonormal eigenvectors ✓")
print("  3. Spectral decomposition H = ΣλₙPₙ ✓")
print("  4. Functional calculus e^{iHt} via spectral decomposition ✓")

In [ ]:
\
# Projection operators and quantum measurement simulation

# Consider a 3-level quantum system (qutrit)
# State: |ψ⟩ = α|0⟩ + β|1⟩ + γ|2⟩

# Define a random normalized state
np.random.seed(42)
psi = np.random.randn(3) + 1j * np.random.randn(3)
psi = psi / np.linalg.norm(psi)

print("Quantum Measurement with Projection Operators")
print("=" * 55)
print(f"State |ψ⟩ = {psi}")
print(f"||ψ|| = {np.linalg.norm(psi):.10f}")

# Basis states
e0 = np.array([1, 0, 0], dtype=complex)
e1 = np.array([0, 1, 0], dtype=complex)
e2 = np.array([0, 0, 1], dtype=complex)
basis = [e0, e1, e2]

# Projection operators P_n = |n⟩⟨n|
print("\nProjection operators and measurement probabilities:")
P_sum = np.zeros((3, 3), dtype=complex)
for n, en in enumerate(basis):
    P_n = np.outer(en, en.conj())
    prob = np.real(psi.conj() @ P_n @ psi)
    projected = P_n @ psi
    print(f"  P_{n} |ψ⟩ → prob = |⟨{n}|ψ⟩|² = {prob:.6f}")
    P_sum += P_n

    # Verify projection properties
    assert np.allclose(P_n @ P_n, P_n), f"P_{n}² ≠ P_{n}"
    assert np.allclose(P_n, P_n.conj().T), f"P_{n} not Hermitian"

print(f"\nCompleteness: Σ P_n = I: {np.allclose(P_sum, np.eye(3))}")
print(f"Probabilities sum to 1: {sum(abs(psi[n])**2 for n in range(3)):.10f}")

# Simulate measurement outcomes
n_measurements = 10000
probs = np.abs(psi)**2
outcomes = np.random.choice([0, 1, 2], size=n_measurements, p=probs)
counts = np.bincount(outcomes, minlength=3)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of theoretical vs measured
ax = axes[0]
x_pos = np.arange(3)
width = 0.35
ax.bar(x_pos - width/2, probs, width, label='Theoretical $|c_n|^2$', color='steelblue')
ax.bar(x_pos + width/2, counts/n_measurements, width, label=f'Measured ({n_measurements} trials)',
       color='coral')
ax.set_xlabel("Outcome $|n\\rangle$", fontsize=12)
ax.set_ylabel("Probability", fontsize=12)
ax.set_title("Quantum Measurement Statistics", fontsize=13)
ax.set_xticks(x_pos)
ax.set_xticklabels([f'$|{n}\\rangle$' for n in range(3)])
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Bloch-like visualization: show |ψ⟩ components
ax2 = axes[1]
amplitudes = np.abs(psi)
phases = np.angle(psi)
colors = plt.cm.hsv(phases / (2*np.pi) + 0.5)

bars = ax2.bar(range(3), amplitudes, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_xlabel("Basis state", fontsize=12)
ax2.set_ylabel("$|c_n|$", fontsize=12)
ax2.set_title("State Amplitudes (color = phase)", fontsize=13)
ax2.set_xticks(range(3))
ax2.set_xticklabels([f'$|{n}\\rangle$\nφ={phases[n]:.2f}' for n in range(3)])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Measurement outcomes follow Born's rule: P(n) = |⟨n|ψ⟩|²")
print("Completeness relation ΣPₙ = I ensures probabilities sum to 1.")

\
## 6. QM Connection: This IS Quantum Mechanics

The mathematical framework of Hilbert spaces *is* the language of quantum mechanics.
The correspondence is exact:

| Mathematics | Quantum Mechanics |
|-------------|------------------|
| Hilbert space $\mathcal{H}$ | State space of the system |
| Unit vector $|\psi\rangle \in \mathcal{H}$ | Pure quantum state |
| Self-adjoint operator $\hat{A}$ | Observable (energy, position, spin) |
| Eigenvalue $\lambda$ of $\hat{A}$ | Possible measurement outcome |
| $|\langle\lambda|\psi\rangle|^2$ | Probability of outcome $\lambda$ |
| Unitary operator $\hat{U}(t)$ | Time evolution |
| Inner product $\langle\phi|\psi\rangle$ | Transition amplitude |
| Tensor product $\mathcal{H}_1 \otimes \mathcal{H}_2$ | Composite systems |

### The Postulates of Quantum Mechanics

1. **States** are vectors in a Hilbert space
2. **Observables** are self-adjoint operators
3. **Measurement** yields eigenvalues with probability $|\langle\lambda|\psi\rangle|^2$
4. **After measurement**, state collapses to eigenstate $|\lambda\rangle$
5. **Time evolution** follows $i\hbar\frac{d}{dt}|\psi\rangle = \hat{H}|\psi\rangle$

Every concept in this notebook maps directly to a physical principle.

In [ ]:
\
# Quantum Harmonic Oscillator: the complete Hilbert space picture
# Eigenstates, operators, and time evolution

from math import factorial
from scipy.special import hermite as physicists_hermite
from scipy.linalg import expm

# Construct the QHO in a truncated Hilbert space (N dimensions)
N = 30  # truncation

# Creation and annihilation operators in Fock basis
a = np.zeros((N, N), dtype=complex)       # annihilation
a_dag = np.zeros((N, N), dtype=complex)   # creation
for n in range(N-1):
    a[n, n+1] = np.sqrt(n+1)
    a_dag[n+1, n] = np.sqrt(n+1)

# Number operator: N_hat = a†a
N_hat = a_dag @ a

# Hamiltonian: H = ℏω(N + 1/2)
omega = 1.0
H = omega * (N_hat + 0.5 * np.eye(N))

# Position and momentum operators: x = (a + a†)/√2, p = i(a† - a)/√2
x_op = (a + a_dag) / np.sqrt(2)
p_op = 1j * (a_dag - a) / np.sqrt(2)

print("Quantum Harmonic Oscillator in Fock Space")
print("=" * 55)

# Verify commutation relations (in truncated Fock space, last element deviates)
commutator_aa = a @ a_dag - a_dag @ a
commutator_xp = x_op @ p_op - p_op @ x_op
# Check subspace excluding truncation boundary
M = N - 1  # exclude last row/col (truncation artifact)
print(f"[a, a†] = I (first {M}x{M} block): {np.allclose(commutator_aa[:M,:M], np.eye(M))}")
print(f"[x, p] = iI (first {M}x{M} block): {np.allclose(commutator_xp[:M,:M], 1j * np.eye(M))}")

# Energy eigenvalues
eigenvalues = np.diag(H).real
print(f"\nEnergy levels E_n = ℏω(n+1/2):")
for n in range(6):
    print(f"  E_{n} = {eigenvalues[n]:.4f} (expected {omega*(n+0.5):.4f})")

# Uncertainty principle: Δx·Δp ≥ ℏ/2 for each eigenstate
print("\nUncertainty principle ΔxΔp ≥ 1/2:")
for n in range(6):
    state = np.zeros(N)
    state[n] = 1.0
    x_avg = np.real(state @ x_op @ state)
    x2_avg = np.real(state @ x_op @ x_op @ state)
    p_avg = np.real(state @ p_op @ state)
    p2_avg = np.real(state @ p_op @ p_op @ state)
    dx = np.sqrt(x2_avg - x_avg**2)
    dp = np.sqrt(p2_avg - p_avg**2)
    print(f"  |{n}⟩: Δx={dx:.4f}, Δp={dp:.4f}, ΔxΔp={dx*dp:.4f} (≥ 0.5)")

# Time evolution of a coherent state |α⟩
alpha = 2.0
coherent = np.zeros(N, dtype=complex)
for n in range(N):
    coherent[n] = np.exp(-abs(alpha)**2/2) * alpha**n / np.sqrt(float(factorial(n)))

# Evolve and compute ⟨x⟩(t)
times = np.linspace(0, 4*np.pi, 200)
x_expect = []
for t in times:
    U = expm(-1j * H * t)
    psi_t = U @ coherent
    x_expect.append(np.real(psi_t.conj() @ x_op @ psi_t))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Energy levels and wavefunctions
ax = axes[0]
x_grid = np.linspace(-6, 6, 500)
for n in range(5):
    # QHO wavefunction: ψ_n(x) = (mω/πℏ)^{1/4} (1/√(2^n n!)) H_n(x) e^{-x²/2}
    Hn = physicists_hermite(n)
    psi_n = (1/np.pi**0.25) * (1/np.sqrt(2**n * factorial(n))) * Hn(x_grid) * np.exp(-x_grid**2/2)
    # Offset by energy level for visualization
    E_n = omega * (n + 0.5)
    ax.fill_between(x_grid, E_n, E_n + 2*psi_n**2, alpha=0.5, label=f'$|{n}\\rangle$')
    ax.hlines(E_n, -6, 6, colors='gray', linewidth=0.5, linestyle='--')

# Potential
ax.plot(x_grid, 0.5 * omega * x_grid**2, 'k-', lw=2, label='$V(x)=\\frac{1}{2}\\omega x^2$')
ax.set_xlim(-5, 5)
ax.set_ylim(0, 6)
ax.set_xlabel("$x$", fontsize=12)
ax.set_ylabel("Energy", fontsize=12)
ax.set_title("QHO Wavefunctions $|\\psi_n(x)|^2$", fontsize=13)
ax.legend(fontsize=9, loc='upper right')
ax.grid(True, alpha=0.2)

# Right: Coherent state time evolution
ax2 = axes[1]
ax2.plot(times/(2*np.pi), x_expect, 'b-', lw=2)
x_classical = np.sqrt(2) * alpha * np.cos(omega * times)
ax2.plot(times/(2*np.pi), x_classical, 'r--', lw=1.5, label='Classical')
ax2.set_xlabel("$t / (2\\pi/\\omega)$", fontsize=12)
ax2.set_ylabel("$\\langle x \\rangle (t)$", fontsize=12)
ax2.set_title(f"Coherent State $|\\alpha={alpha}\\rangle$ Evolution", fontsize=13)
ax2.legend(['Quantum $\\langle\\hat{x}\\rangle$', 'Classical $x(t)$'], fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Left: QHO eigenstates — quantized energy levels with wavefunctions.")
print("Right: Coherent state ⟨x⟩(t) exactly tracks classical trajectory — Ehrenfest's theorem!")

In [ ]:
\
# Spectral decomposition applied: quantum gates and state tomography

# Hadamard gate: H = (σ_x + σ_z)/√2
H_gate = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)

# Phase gate: S = diag(1, i)
S_gate = np.array([[1, 0], [0, 1j]], dtype=complex)

# T gate: T = diag(1, e^{iπ/4})
T_gate = np.array([[1, 0], [0, np.exp(1j*np.pi/4)]], dtype=complex)

# CNOT gate (2-qubit)
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]], dtype=complex)

print("Quantum Gate Spectral Decomposition")
print("=" * 55)

for name, gate in [("Hadamard H", H_gate), ("Phase S", S_gate), ("T-gate", T_gate)]:
    eigenvalues, eigvecs = np.linalg.eig(gate)
    print(f"\n{name}:")
    print(f"  Matrix:\n    {gate[0]}\n    {gate[1]}")
    print(f"  Eigenvalues: {eigenvalues}")
    print(f"  Unitary: {np.allclose(gate.conj().T @ gate, np.eye(2))}")

    # Reconstruct from spectral decomposition
    gate_recon = np.zeros((2,2), dtype=complex)
    for lam, v in zip(eigenvalues, eigvecs.T):
        gate_recon += lam * np.outer(v, v.conj())
    print(f"  Spectral reconstruction matches: {np.allclose(gate, gate_recon)}")

# Demonstrate: create Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2
print("\nBell State Creation: |00⟩ → H⊗I → CNOT → |Φ+⟩")
psi_00 = np.array([1, 0, 0, 0], dtype=complex)
H_tensor_I = np.kron(H_gate, np.eye(2))
psi_after_H = H_tensor_I @ psi_00
psi_bell = CNOT @ psi_after_H

print(f"  |00⟩ = {psi_00}")
print(f"  (H⊗I)|00⟩ = {np.round(psi_after_H, 4)}")
print(f"  CNOT(H⊗I)|00⟩ = {np.round(psi_bell, 4)}")
print(f"  This is |Φ+⟩ = (|00⟩+|11⟩)/√2: {np.allclose(psi_bell, np.array([1,0,0,1])/np.sqrt(2))}")

# Entanglement verification: compute reduced density matrix
rho = np.outer(psi_bell, psi_bell.conj())
# Partial trace over qubit 2: ρ_A = Tr_B(ρ)
rho_A = np.zeros((2,2), dtype=complex)
for j in range(2):
    for k in range(2):
        for l in range(2):
            rho_A[j, k] += rho[2*j+l, 2*k+l]

print(f"\n  Reduced density matrix ρ_A:")
print(f"    {rho_A[0]}")
print(f"    {rho_A[1]}")
eigenvalues_rho = np.linalg.eigvalsh(rho_A)
von_neumann_entropy = -sum(lam * np.log2(lam) for lam in eigenvalues_rho if lam > 1e-10)
print(f"  Von Neumann entropy S(ρ_A) = {von_neumann_entropy:.4f} bits (max for 2D = 1.0)")
print(f"  ρ_A = I/2 (maximally mixed): {np.allclose(rho_A, np.eye(2)/2)}")
print(f"  → Maximally entangled state confirmed!")

\
## Summary

| Topic | Key Result |
|-------|-----------|
| Metric spaces | Distance, completeness, Cauchy sequences |
| $L^2$ space | Square-integrable functions with inner product $\langle f,g\rangle = \int f^*g$ |
| Fourier basis | Complete orthonormal set; Parseval: $\|f\|^2 = \Sigma|c_n|^2$ |
| Operators | Linear maps on $\mathcal{H}$; Hermitian, unitary, projectors |
| Spectral theorem | $\hat{A} = \sum \lambda_n |\lambda_n\rangle\langle\lambda_n|$ |
| Functional calculus | $f(\hat{A}) = \sum f(\lambda_n) |\lambda_n\rangle\langle\lambda_n|$ |
| QM = Hilbert space | States ↔ vectors, observables ↔ operators, probabilities ↔ $|c_n|^2$ |

This is the mathematical foundation upon which **all** of quantum mechanics is built.
Every concept from Year 1 onwards will use this language.

**Next:** Month 10 — Scientific Computing

---
*SIIEA Quantum Engineering Curriculum — CC BY-NC-SA 4.0*